In [1]:
from config import api_keys
from citipy import citipy

import random
import requests

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt


In [2]:
lat_zone = np.arange(-80,80,15)
lon_zone = np.arange(-200,200,15)


cities_df = pd.DataFrame()

cities_df["Latitude"] = ""
cities_df["Longitude"] = ""

cities = {}

for x in lat_zone:
    for y in lon_zone:
        x_values = list(np.arange(x,x+15,0.01))
        y_values = list(np.arange(y,y+15,0.01))
        lats = random.sample(x_values,50)
        lons = random.sample(y_values,50)
        lat_samples = [(x+dec_lat) for dec_lat in lats]
        lon_samples = [y+dec_lon for dec_lon in lons]
        cities_df = cities_df.append(pd.DataFrame.from_dict({"Latitude":lat_samples,
                                       "Longitude":lon_samples}))
cities_df = cities_df.reset_index(drop=True)
cities_df.shape

(14850, 2)

In [3]:
cities_df["Closest City name"] = ""
cities_df["Closest Country code"] = ""
for index,row in cities_df.iterrows():
    city = citipy.nearest_city(row["Latitude"],row["Longitude"])
    cities_df.set_value(index,"Closest City name",city.city_name)
    cities_df.set_value(index,"Closest Country code",city.country_code)



In [4]:
# Dropping random lat and lon samples from dataframe

clean_cities_df = cities_df.drop(['Latitude', 'Longitude'],axis=1)
clean_cities_df
clean_cities_df = clean_cities_df.drop_duplicates()

clean_cities_df.shape

(899, 2)

In [5]:
selected_cities = clean_cities_df.sample(100)

In [6]:
selected_cities = selected_cities.reset_index(drop=True)
selected_cities

,Closest City name,Closest Country code
0,llangefni,gb
1,atar,mr
2,khatanga,ru
3,timucuy,mx
4,puerto ayora,ec
5,sao filipe,cv
6,natchez,us
7,dikson,ru
8,haimen,cn
9,codrington,ag


In [8]:
base_url = "http://api.openweathermap.org/data/2.5/weather"

app_id=api_keys["open_weather_map_common"]

params = { "appid" :app_id,"units":"metric" }




In [9]:
# SAMPLE JSON RESPONSE - 
#{"coord":{"lon":-0.13,"lat":51.51},
#"weather":[{"id":300,"main":"Drizzle","description":"light intensity drizzle","icon":"09d"}],
#"base":"stations",
#"main":{"temp":280.32,"pressure":1012,"humidity":81,"temp_min":279.15,"temp_max":281.15},
#"visibility":10000,
# "wind":{"speed":4.1,"deg":80},
# "clouds":{"all":90},
# "dt":1485789600,
# "sys":{"type":1,"id":5091,"message":0.0103,"country":"GB","sunrise":1485762037,"sunset":1485794875},
# "id":2643743,"name":"London","cod":200}

def encrypt_key(input_url):
    return input_url[0:53]+"<YourKey>"+input_url[85:]

for index,row in selected_cities.iterrows():
    params["q"] =f'{row["Closest City name"]},{row["Closest Country code"]}'
    print(f"Retrieving weather information for {params['q']}")
    city_weather_resp = requests.get(base_url,params)
    print(encrypt_key(city_weather_resp.url))
    city_weather_resp  = city_weather_resp.json()
    selected_cities.set_value(index,"Latitude",city_weather_resp.get("coord",{}).get("lat"))
    selected_cities.set_value(index,"Longitude",city_weather_resp.get("coord",{}).get("lon"))
    selected_cities.set_value(index,"Temperature",city_weather_resp.get("main",{}).get("temp_max"))
    selected_cities.set_value(index,"Wind speed",city_weather_resp.get("wind",{}).get("speed"))
    selected_cities.set_value(index,"Humidity",city_weather_resp.get("main",{}).get("humidity"))
    selected_cities.set_value(index,"Cloudiness",city_weather_resp.get("clouds",{}).get("all"))


Retrieving weather information for llangefni,gb
http://api.openweathermap.org/data/2.5/weather?appid=<YourKey>&units=metric&q=llangefni%2Cgb
Retrieving weather information for atar,mr
http://api.openweathermap.org/data/2.5/weather?appid=<YourKey>&units=metric&q=atar%2Cmr
Retrieving weather information for khatanga,ru
http://api.openweathermap.org/data/2.5/weather?appid=<YourKey>&units=metric&q=khatanga%2Cru
Retrieving weather information for timucuy,mx
http://api.openweathermap.org/data/2.5/weather?appid=<YourKey>&units=metric&q=timucuy%2Cmx
Retrieving weather information for puerto ayora,ec
http://api.openweathermap.org/data/2.5/weather?appid=<YourKey>&units=metric&q=puerto+ayora%2Cec
Retrieving weather information for sao filipe,cv
http://api.openweathermap.org/data/2.5/weather?appid=<YourKey>&units=metric&q=sao+filipe%2Ccv
Retrieving weather information for natchez,us
http://api.openweathermap.org/data/2.5/weather?appid=<YourKey>&units=metric&q=natchez%2Cus
Retrieving weather inform

http://api.openweathermap.org/data/2.5/weather?appid=<YourKey>&units=metric&q=port+hedland%2Cau
Retrieving weather information for kharod,in
http://api.openweathermap.org/data/2.5/weather?appid=<YourKey>&units=metric&q=kharod%2Cin
Retrieving weather information for pafos,cy
http://api.openweathermap.org/data/2.5/weather?appid=<YourKey>&units=metric&q=pafos%2Ccy
Retrieving weather information for zyukayka,ru
http://api.openweathermap.org/data/2.5/weather?appid=<YourKey>&units=metric&q=zyukayka%2Cru
Retrieving weather information for birjand,ir
http://api.openweathermap.org/data/2.5/weather?appid=<YourKey>&units=metric&q=birjand%2Cir
Retrieving weather information for chase,ca
http://api.openweathermap.org/data/2.5/weather?appid=<YourKey>&units=metric&q=chase%2Cca
Retrieving weather information for ruwi,om
http://api.openweathermap.org/data/2.5/weather?appid=<YourKey>&units=metric&q=ruwi%2Com
Retrieving weather information for bima,id
http://api.openweathermap.org/data/2.5/weather?appid=

In [10]:
selected_cities = selected_cities.dropna()

selected_cities.describe()

,Latitude,Longitude,Temperature,Wind speed,Humidity,Cloudiness
count,86.000000,86.000000,86.000000,86.000000,86.000000,86.000000
mean,23.087674,31.512093,12.746744,3.513023,77.627907,45.697674
std,30.935795,90.000839,15.667408,2.732090,21.442150,36.273818
min,-41.870000,-125.240000,-28.320000,0.530000,10.000000,0.000000
25%,-2.665000,-59.110000,2.297500,1.447500,64.000000,8.000000
50%,24.955000,38.510000,16.020000,3.050000,82.500000,44.000000
75%,50.710000,115.162500,25.057500,4.942500,93.000000,76.000000
max,73.510000,178.420000,34.410000,15.730000,100.000000,100.000000
